# Setup


In [ ]:
# google_cloud_vision.ipynb, a Jupyter notebook for analyzing images using the Google Cloud Vision API
version = '0.1.0'
created = '2023-03-27'

# (c) 2023 Vanderbilt University. This program is released under a GNU General Public License v3.0 http://www.gnu.org/licenses/gpl-3.0
# Author: Steve Baskauf
# For more information, see https://github.com/HeardLibrary/linked-data/tree/master/image_analysis

# This script carries out three major tasks:
# 1. It downloads images from the Vanderbilt University Libraries IIIF server at a standard resolution of 1000 pixels in 
# the shortest dimension. If the image is already smaller than 1000 pixels in the shortest dimension, then the image is
# downloaded at the original resolution.
# 2. It analyzes the images using the Google Cloud Vision API using the FACE_DETECTION, LABEL_DETECTION, OBJECT_LOCALIZATION, and
# TEXT_DETECTION features. The results are saved to CSV files.
# 3. Optionally, it can generate an IIIF annotation file for each image that can be used to display the results of the
# object localization analysis of the image.

# -----------------------------------------
# Version 0.1.0 change notes (2023-03-27):
# - Initial version
# -----------------------------------------

import pandas as pd
import requests
import shutil # high-level file operations
from PIL import Image
import os
import shutil

computer_username = 'baskausj' # baskauf for home computer, baskausj for work computer

# Load the image data into a dataframe
base_path = '/Users/'+ computer_username + '/github/vandycite/gallery_buchanan/image_analysis/'
download_path = '/Users/'+ computer_username + '/Downloads/'
metadata_path = '/Users/'+ computer_username + '/github/vandycite/gallery_buchanan/'
past_upload_metadata_path = '/Users/'+ computer_username + '/github/vandycite/gallery_works/image_upload/'


# Determine which images are prints that have not yet been analyzed.

Our priority as of 2023-09-15 is to start analyzing works that might have text on them

In [ ]:
# Load the metadata on all works
metadata_df = pd.read_csv(metadata_path + 'works_multiprop.csv', dtype=str, na_filter=False)
# Set the index to the work qid
metadata_df = metadata_df.set_index('qid')

# Load the works classification metadata
classification_df = pd.read_csv(metadata_path + 'works_classification.csv', dtype=str, na_filter=False)
# Set the index to the work qid
classification_df = classification_df.set_index('qid')

# Join the two dataframes based on the qid value
metadata_df = metadata_df.join(classification_df, how='left')

# Filter for rows that have a value of "print" in the "type" column
metadata_df = metadata_df[metadata_df['type'] == 'print']

# Create a list of accession numbers from the inventory_number column
accession_list = metadata_df['inventory_number'].tolist()


# The combined_images.csv file is a list of all images that have been downloaded from the IIIF server for analysis.

# Load the combined_images.csv file
combined_images_df = pd.read_csv(base_path + 'combined_images.csv', dtype=str, na_filter=False)
# Creat a list of accession_number values
combined_images_list = combined_images_df['accession_number'].tolist()

# Create a list of accession numbers that have not yet been downloaded
download_list = [x for x in accession_list if x not in combined_images_list]

print(len(download_list))

Now we need to sort through the image metadata to figure out which of these accessioned works have images that have been classified as "primary" (i.e. they have a single "best" image suitable for putting into the IIIF server). As of 2023-09-15, not all of the classifying has been finished, but most of the unclassified ones were either 3D works or works that are parts of sets. So there should be at least a thousand that are ready to do now.

In [ ]:
# Get the data for all images (not just the downloaded ones).
all_image_data_df = pd.read_csv(past_upload_metadata_path + 'images_use_combined_images.csv', dtype=str, na_filter=False)
all_image_data_df.head()

# Loop through the list of download accession numbers and pull the rows from the all_image_data_df dataframe
# that match the accession number. The local_identifier column in the all_image_data_df dataframe has the accession number.

# Filter the all_image_data_df dataframe to only include rows that have a value of "primary" in the rank column.
all_image_data_df = all_image_data_df[all_image_data_df['rank'] == 'primary']

# Create an empty dataframe to hold the rows pulled from the all_image_data_df dataframe. Use the same column names.
download_image_data_df = pd.DataFrame(columns=all_image_data_df.columns)

for accession_number in download_list:
    # Get the rows from the all_image_data_df dataframe that match the accession number
    download_image_data_df = download_image_data_df.append(all_image_data_df.loc[all_image_data_df['local_identifier'] == accession_number], ignore_index=True)

print(len(download_image_data_df))

# Save the download_image_data_df dataframe to a CSV file
download_image_data_df.to_csv(base_path + 'download_image_data.csv', index=False)



We need to copy these images to another directory so that they can be uploaded to the IIIF server. If they are TIFFs, they should be converted to tiled pyramidal tiffs first.

In [ ]:
def image_magick_convert_tiff(in_path: str, out_path: str, log_path: str):
    """Convert a TIFF file to a pyramidal tiled TIFF file using ImageMagick.

    Parameters
    ----------
    in_path: path to the input TIFF file
    out_path: path to the output pyramidal tiled TIFF file
    log_path: path to a text log file to which warnings, errors, and other output will be appended.

    Based on practical experience, most errors can be ignored."""
    # Note: need to enclose file paths in quotes because filenames sometimes include spaces.
    # 256x256 is the tile size and seems to be a good choice for IIIF viewers. 
    # The depth 8 setting should not be changed as it is necessary to view all of the tiles in Gimp and 
    # for preview to work properly in Mac Preview.
    # The number of images included in the pyramid will depend on the filesize of the original image.
    command_string = 'convert "' + in_path + '" -define tiff:tile-geometry=256x256 -depth 8 ptif:"' + out_path + '" 2>> "' + log_path + '"'
    #print(command_string)
    os.system(command_string)

# Set the path to the directory containing the TIFF files to be converted
# NOTE: this script will convert all TIFF files in the directory, and will ignore other filetypes. 
# The path should end with a slash.
in_base_dir = '/Users/baskausj/gallery_digital_image_archive/'

# Set the path to the directory where the converted files will be written. The path should end with a slash.
out_dir = '/Users/baskausj/gallery_pyramidal_tiffs/prints_analysis/'

# Open the download_image_data.csv file
download_image_data_df = pd.read_csv(base_path + 'download_image_data.csv', dtype=str, na_filter=False)

# Set the path to the log file
log_path = out_dir + 'log.txt'


# Loop through the files download_image_data_df dataframe
for index, row in download_image_data_df.iterrows():
    # Get the filename from the local_filename column
    filename = row['local_filename']
    print(index, filename)

    # Set the path to the input file
    in_path = in_base_dir + row['subdir'] + '/' + row['local_filename']
    # Set the path to the output file
    out_path = out_dir + filename
    # Write the file name to the log file
    with open(log_path, 'a') as log_file:
        log_file.write(filename + '\n')

    # Check to see if the file is a TIFF file
    if filename.lower().endswith('.tif') or filename.lower().endswith('.tiff'):
        # Convert the file
        image_magick_convert_tiff(in_path, out_path, log_path)
    else:
        # Copy the file
        shutil.copyfile(in_path, out_path)

print('done')


# Completing analysis of all images (2024-03-19)

It doesn't look like I was careful about consistently saving the same files that record which images have been analyzed. However, there are two files that contain the accession numbers of images that have been done: `combined_images.csv` contains the round 1 data and `accession_dimensions.csv` contains the round 2 data. So the accession numbers in those files should be excluded from the final analysis.


In [ ]:
# Load the combined_images.csv file
round1_df = pd.read_csv(base_path + 'combined_images.csv', dtype=str, na_filter=False)

# Load the accession_dimensions.csv file
round2_df = pd.read_csv(base_path + 'accession_dimensions.csv', dtype=str, na_filter=False)

# Create a list of accession numbers from the accession_number column of the two dataframes
accession_number_list = round1_df['accession_number'].tolist() + round2_df['accession_number'].tolist()

# Get the data for all images (not just the downloaded ones).
all_image_data_df = pd.read_csv(past_upload_metadata_path + 'images_use_combined_images.csv', dtype=str, na_filter=False)
all_image_data_df.head()

# The local_identifier column in the all_image_data_df dataframe has the accession number.

# Filter the all_image_data_df dataframe to only include rows that have a value of "primary" in the rank column.
all_image_data_df = all_image_data_df[all_image_data_df['rank'] == 'primary']

# Create an empty dataframe to hold the rows pulled from the all_image_data_df dataframe. Use the same column names.
download_image_data_df = pd.DataFrame(columns=all_image_data_df.columns)

for accession_number in accession_number_list:
    # remove the row with the accession_number from the DataFrame
    all_image_data_df = all_image_data_df[all_image_data_df.local_identifier != accession_number]
    
print(len(all_image_data_df))

# Save the download_image_data_df dataframe to a CSV file
download_image_data_df.to_csv(base_path + 'images_to_process.csv', index=False)



# Download the images to a local directory

NOTE: this only needs to be done once for a given set of images. Once they are loaded into the bucket it doesn't need to be run again.

This code uses a list of accession numbers (found as a column in a CSV file) to generate IIIF Image API (v2) URLs for JPEG images that are 1000 pixels in the shortest dimension, then download them into a local directory.

After generating and downloading the images, they need to be uploaded to the Google Cloud bucket used in the Vision analysis.

NOTE: There are two versions of this code. Only one or the other of the following cells should be run!

The *first version* is for images that were already loaded into the IIIF server during the Commons upload and therefore they had manifests already generated. 

The *second version* also generates smaller JPEGs using the Image API, but gets its metadata from the `download_image_data.csv` file output from a cell above rather than from manifests. The files are in a single directory rather than in directories organized by year.


In [ ]:
# !!!!!!!!!!!!!
# First version
# !!!!!!!!!!!!!

# Load the source image data into a dataframe
source_image_dataframe = pd.read_csv(base_path + 'combined_images.csv', dtype=str)
# Set the commons_id column as the index
source_image_dataframe = source_image_dataframe.set_index('commons_id')

#source_image_dataframe.head()

# Import CSV data as a dataframe.
accession_dataframe = pd.read_csv(base_path + 'accession_numbers_to_analyze.csv', dtype=str)

# Test with a single row
#accession_dataframe = accession_dataframe.head(1)


# Create a dataframe to hold the accession numbers and dimensions
accession_dimensions_dataframe = pd.DataFrame(columns=['accession_number', 'height', 'width'])

bad_image_list = []

# Loop through the dataframe rows and download the images.
for index, row in accession_dataframe.iterrows():
    accession_number = row['accession_number']
    print(accession_number)
    
    # Look up the image data in the source image dataframe.
    # In cases where there are two images, we want the primary image.
    image_series = source_image_dataframe.loc[(source_image_dataframe['accession_number'] == accession_number) & (source_image_dataframe['rank'] == 'primary')]
    manifest_url = image_series['iiif_manifest'][0]

    # get the manifest from the manifest url
    manifest = requests.get(manifest_url).json()
    #print(json.dumps(manifest, indent=2))
    service_url = manifest['sequences'][0]['canvases'][0]['images'][0]['resource']['service']['@id']
    # Because of the error in original manifests, replace version 3 with version 2 in the URL.
    #service_url = service_url.replace('/3/', '/2/') # This is no longer needed because the manifests have been fixed.
    #print('service_url', service_url)

    # Determine the maximum and minimum dimensions of the image.
    height = image_series['height'][0]
    #print('height', height)
    width = image_series['width'][0]
    #print('width', width)
    shortest_dimension = min(int(height), int(width))
    longest_dimension = max(int(height), int(width))
    #print('shortest_dimension', shortest_dimension)

    # We want to know what the largest dimension needs to be for the shortest dimension to be 1000 pixels.
    # If that calculation makes the longest dimension longer than the actual longest dimension, 
    # then we want to use the actual longest dimension.
    # If the shortest dimension is already less than 1000 pixels, then we will just use the longest dimension as is.
    if shortest_dimension > 1000:
        size = int(1000 * (longest_dimension / shortest_dimension))
        if size > longest_dimension:
            size = longest_dimension
    else:
        size = longest_dimension
    #print('size', size)

    # construct the image url using the "!" size option, that keeps the aspect ratio but sizes to the maximum dimension.
    image_url = service_url + '/full/!' + str(size) + ',' + str(size) + '/0/default.jpg'
    print('image_url', image_url)
    print()
        
    # retrieve the image from the IIIF server
    image_object = requests.get(image_url, stream=True).raw

    # Save the image to a file
    with open(download_path + 'google_vision_images/' + accession_number + '.jpg', 'wb') as out_file:
        shutil.copyfileobj(image_object, out_file)
        # Force the file to be written to disk
        out_file.flush()
    
    # Find the image dimensions
    # Open the image file from disk
    with open(download_path + 'google_vision_images/' + accession_number + '.jpg', 'rb') as image_file:
        reduced_width = 0
        reduced_height = 0
        try:
            image = Image.open(image_file)
            reduced_width, reduced_height = image.size
            #print('reduced_width', reduced_width)
            #print('reduced_height', reduced_height)
        except:
            print('bad image')
            bad_image_list.append(accession_number)
            continue
    
    # Display the image
    #image.show()

    # Add the accession number and dimensions to the dataframe
    accession_dimensions_dataframe = accession_dimensions_dataframe.append({'accession_number': accession_number, 'max_height': height, 'max_width': width, 'height': reduced_height, 'width': reduced_width}, ignore_index=True)

# Save the dataframe to a CSV file
accession_dimensions_dataframe.to_csv(base_path + 'accession_dimensions.csv', index=False)

print('bad_image_list', bad_image_list)

print('done')


In [ ]:
# !!!!!!!!!!!!!
# Second version
# !!!!!!!!!!!!!

# Load the source image data into a dataframe
source_image_dataframe = pd.read_csv(base_path + 'download_image_data.csv', dtype=str, na_filter=False)

# Test with a single row
#source_image_dataframe = source_image_dataframe.head(1)


# Create a dataframe to hold the accession numbers and dimensions
accession_dimensions_dataframe = pd.DataFrame(columns=['accession_number', 'height', 'width'])

bad_image_list = []

# Loop through the dataframe rows and download the images.
for index, row in source_image_dataframe.iterrows():
    accession_number = row['local_identifier']
    print(accession_number)

    # Determine the maximum and minimum dimensions of the image.
    height = row['height']
    #print('height', height)
    width = row['width']
    #print('width', width)
    shortest_dimension = min(int(height), int(width))
    longest_dimension = max(int(height), int(width))
    #print('shortest_dimension', shortest_dimension)

    # We want to know what the largest dimension needs to be for the shortest dimension to be 1000 pixels.
    # If that calculation makes the longest dimension longer than the actual longest dimension, 
    # then we want to use the actual longest dimension.
    # If the shortest dimension is already less than 1000 pixels, then we will just use the longest dimension as is.
    if shortest_dimension > 1000:
        size = int(1000 * (longest_dimension / shortest_dimension))
        if size > longest_dimension:
            size = longest_dimension
    else:
        size = longest_dimension
    #print('size', size)

    # Construct the service_url from the image API root URL, the storage directory path, and the filename.
    service_url = 'https://iiif.library.vanderbilt.edu/iiif/2/gallery%2Fanalysis%2F' + row['local_filename']

    # construct the image url using the "!" size option, that keeps the aspect ratio but sizes to the maximum dimension.
    image_url = service_url + '/full/!' + str(size) + ',' + str(size) + '/0/default.jpg'
    print('image_url', image_url)
    print()
        
    # retrieve the image from the IIIF server
    image_object = requests.get(image_url, stream=True).raw

    # Save the image to a file
    with open(download_path + 'google_vision_images/' + accession_number + '.jpg', 'wb') as out_file:
        shutil.copyfileobj(image_object, out_file)
        # Force the file to be written to disk
        out_file.flush()
    
    # Find the image dimensions
    # Open the image file from disk
    with open(download_path + 'google_vision_images/' + accession_number + '.jpg', 'rb') as image_file:
        reduced_width = 0
        reduced_height = 0
        try:
            image = Image.open(image_file)
            reduced_width, reduced_height = image.size
            #print('reduced_width', reduced_width)
            #print('reduced_height', reduced_height)
        except:
            print('bad image')
            bad_image_list.append(accession_number)
            continue
    
    # Display the image
    #image.show()

    # Add the accession number and dimensions to the dataframe
    accession_dimensions_dataframe = accession_dimensions_dataframe.append({'accession_number': accession_number, 'max_height': height, 'max_width': width, 'height': reduced_height, 'width': reduced_width}, ignore_index=True)

# Save the dataframe to a CSV file
accession_dimensions_dataframe.to_csv(base_path + 'accession_dimensions.csv', index=False)

print('bad_image_list', bad_image_list)

print('done')


# Google Cloud Vision image analysis


In [ ]:
# This code is part of google_cloud_vision.ipynb
# For licensing and other information, see https://github.com/HeardLibrary/linked-data/tree/master/image_analysis

# Here's the landing page for Google Cloud Vision
# https://cloud.google.com/vision/
# From it you can try the api by dragging and dropping an image into the browser. You can then 
# view the JSON response, which was helpful at first to understand the structure of the response.

# The following tutorial contains critical information about enabling the API and creating a role
# for the service account to allow it access. This is followed by creating a service account key.
# https://cloud.google.com/vision/docs/detect-labels-image-client-libraries

# I didn't actually do this tutorial, but it was useful to understand the order of operations that
# needed to be done prior to writing to the API.
# https://www.cloudskillsboost.google/focuses/2457?parent=catalog&utm_source=vision&utm_campaign=cloudapi&utm_medium=webpage
# Because I'm using the Python client library, the part about setting up the request body was irrelevant. 
# But the stuff about uploading the files to the bucket, making it publicly accessible, etc. was helpful.
import json
import pandas as pd
from typing import List, Dict, Tuple, Any, Optional

# Imports the Google Cloud client library
# Reference for Google Cloud Vision Python client https://cloud.google.com/python/docs/reference/vision/latest
from google.cloud import vision
from google.cloud import vision_v1
from google.cloud.vision_v1 import AnnotateImageResponse

# Import from Google oauth library
from google.oauth2 import service_account

def extract_object_localization_data(accession_number: str, annotation: List[Dict[str, Any]], width: int, height: int) -> Dict[str, Any]:
    """Extract the object localization data from a hit and turn it into a dict to be added as a row in the dataframe."""
    #print('annotation', annotation)
    description = annotation['name']
    score = annotation['score']
    left_x = annotation['boundingPoly']['normalizedVertices'][0]['x']
    top_y = annotation['boundingPoly']['normalizedVertices'][0]['y']
    right_x = annotation['boundingPoly']['normalizedVertices'][2]['x']
    bottom_y = annotation['boundingPoly']['normalizedVertices'][2]['y']
    #print('description', description)
    #print('score', score)
    #print('left_x', left_x)
    #print('top_y', top_y)
    #print('right_x', right_x)
    #print('bottom_y', bottom_y)
    #print()

    row = {'accession_number': accession_number, 'description': description, 'score': score, 'rel_left_x': left_x, 'rel_right_x': right_x, 'rel_top_y': top_y, 'rel_bottom_y': bottom_y, 'abs_left_x': round(left_x * width), 'abs_right_x': round(right_x * width), 'abs_top_y': round(top_y * height), 'abs_bottom_y': round(bottom_y * height)}
    return row

def extract_face_detection_data(accession_number: str, annotation: List[Dict[str, Any]], width: int, height: int) -> Dict[str, Any]:
    """Extract the face detection data from a hit and turn it into a dict to be added as a row in the dataframe."""
    score = annotation['detectionConfidence']
    left_x = annotation['boundingPoly']['vertices'][0]['x']
    top_y = annotation['boundingPoly']['vertices'][0]['y']
    right_x = annotation['boundingPoly']['vertices'][2]['x']
    bottom_y = annotation['boundingPoly']['vertices'][2]['y']
    roll_angle = annotation['rollAngle']
    pan_angle = annotation['panAngle']
    tilt_angle = annotation['tiltAngle']
    landmarking_confidence = annotation['landmarkingConfidence']
    joy_likelihood = annotation['joyLikelihood']
    sorrow_likelihood = annotation['sorrowLikelihood']
    anger_likelihood = annotation['angerLikelihood']
    surprise_likelihood = annotation['surpriseLikelihood']
    under_exposed_likelihood = annotation['underExposedLikelihood']
    blurred_likelihood = annotation['blurredLikelihood']
    headwear_likelihood = annotation['headwearLikelihood']

    row = {'accession_number': accession_number, 'score': score, 
           'rel_left_x': left_x / width, 'rel_right_x': right_x / width, 'rel_top_y': top_y / height, 'rel_bottom_y': bottom_y /height,
           'abs_left_x': left_x, 'abs_right_x': right_x, 'abs_top_y': top_y, 'abs_bottom_y': bottom_y,
           'roll_angle': roll_angle, 'pan_angle': pan_angle, 'tilt_angle': tilt_angle,
           'landmarking_confidence': landmarking_confidence, 'joy_likelihood': joy_likelihood, 
           'sorrow_likelihood': sorrow_likelihood, 'anger_likelihood': anger_likelihood, 
           'surprise_likelihood': surprise_likelihood, 'under_exposed_likelihood': under_exposed_likelihood,
           'blurred_likelihood': blurred_likelihood, 'headwear_likelihood': headwear_likelihood}
    return row

def extract_label_detection_data(accession_number: str, annotation: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Extract the label detection data from a hit and turn it into a dict to be added as a row in the dataframe."""
    mid = annotation['mid']
    description = annotation['description']
    score = annotation['score']
    topicality = annotation['topicality']
    row = {'accession_number': accession_number, 'mid': mid, 'description': description, 'score': score, 'topicality': topicality}
    return row

def extract_text_detection_data(accession_number: str, annotation: List[Dict[str, Any]], width: int, height: int) -> Dict[str, Any]:
    """Extract the text detection data from a hit and turn it into a dict to be added as a row in the dataframe."""
    locale = annotation['locale']
    description = annotation['description']
    left_x = annotation['boundingPoly']['vertices'][0]['x']
    top_y = annotation['boundingPoly']['vertices'][0]['y']
    right_x = annotation['boundingPoly']['vertices'][2]['x']
    bottom_y = annotation['boundingPoly']['vertices'][2]['y']
    row = {'accession_number': accession_number, 'locale': locale, 'description': description, 
           'rel_left_x': left_x / width, 'rel_right_x': right_x / width, 'rel_top_y': top_y / height, 'rel_bottom_y': bottom_y / height,
           'abs_left_x': left_x, 'abs_right_x': right_x, 'abs_top_y': top_y, 'abs_bottom_y': bottom_y,
           }
    return row

# Customize for your own computer
user_dir = 'baskausj' # Enter your user directory name here
base_path = '/Users/baskausj/github/vandycite/gallery_buchanan/image_analysis/' # Location of the accession number data file
annotations_base_url = 'https://baskaufs.github.io/iiif/baskauf/'

# Set the path to the service account key
key_path = '/Users/' + user_dir + '/image-analysis-376619-193859a33600.json'

# ---------------------------------------------
# Retrieve the service key, create a credentials object, then use it to authenticate and create a `client` object.
# ---------------------------------------------

# Create a credentials object from the service account key
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# API documentation https://cloud.google.com/python/docs/reference/vision/latest/google.cloud.vision_v1.services.image_annotator.ImageAnnotatorClient#methods
# The first two versions have no arguments and the credentials are loaded from the environment variable.
#client = vision.ImageAnnotatorClient()
# Used this specific v1 to get the JSON conversion to work
#client = vision_v1.ImageAnnotatorClient()
# Use this line instead of the one above to load the credentials directly from the file
client = vision_v1.ImageAnnotatorClient(credentials=credentials)

# Load the source data from a CSV. The critical column needed here is the `accession_number` column, since it is the one 
# that was used to construct the image file name for the uploaded test images.
accession_dataframe = pd.read_csv(base_path + 'accession_dimensions.csv', dtype=str)
#accession_dataframe.head()

# -----------------------------------
# Loop through all of the accession numbers and perform the analysis on each of the images.
# -----------------------------------

# I did these 100 at a time and stored the intermediate results in case I had to stop or if the process was interrupted.


# Create a new dataframe to hold the annotations
object_localization_dataframe = pd.DataFrame(columns=['accession_number', 'description', 'score', 'rel_left_x', 'rel_right_x', 'rel_top_y', 'rel_bottom_y', 'abs_left_x', 'abs_right_x', 'abs_top_y', 'abs_bottom_y'])
face_detection_dataframe = pd.DataFrame(columns=['accession_number', 'score', 'rel_left_x', 'rel_right_x', 'rel_top_y', 'rel_bottom_y', 'abs_left_x', 'abs_right_x', 'abs_top_y', 'abs_bottom_y', 'roll_angle', 'pan_angle', 'tilt_angle', 'landmarking, confidence', 'joy_likelihood', 'sorrow_likelihood', 'anger_likelihood', 'surprise_likelihood', 'under_exposed_likelihood', 'blurred_likelihood', 'headwear_likelihood'])
label_detection_dataframe = pd.DataFrame(columns=['accession_number', 'mid', 'description', 'score', 'topicality'])
text_detection_dataframe = pd.DataFrame(columns=['accession_number', 'locale', 'description', 'rel_left_x', 'rel_right_x', 'rel_top_y', 'rel_bottom_y', 'abs_left_x', 'abs_right_x', 'abs_top_y', 'abs_bottom_y'])

# Loop through the dataframe rows and analyze the images.
for index, row in accession_dataframe.iterrows():
    accession_number = row['accession_number']
    print('accession_number', accession_number)
    width = int(row['width'])
    height = int(row['height'])

    # To access the images, they should be stored in a Google Cloud Storage bucket that is set up for public access.
    # It's also possible to use a publicly accessible URL, but that seems to be unreliable.
    # The storage costs for a few images are negligible.

    # Construct the path to the image file
    image_uri = 'gs://vu-gallery/' + accession_number + '.jpg'
    #print('image_uri', image_uri)
    
    # Here is the API documentation for the Feature object.
    # https://cloud.google.com/vision/docs/reference/rest/v1/Feature
    #analysis_type = vision.Feature.Type.FACE_DETECTION
    #analysis_type = vision.Feature.Type.LABEL_DETECTION
    #analysis_type = vision.Feature.Type.OBJECT_LOCALIZATION

    # This API documentation isn't exactly the one for the .annotate_image method, but it's close enough.
    # https://cloud.google.com/vision/docs/reference/rest/v1/projects.images/annotate
    # In particular, it links to the AnnotateImageRequest object, which is what we need to pass to the annotate_image method.
    response = client.annotate_image({
    'image': {'source': {'image_uri': image_uri}},
    'features': [
        {'type_': vision.Feature.Type.OBJECT_LOCALIZATION},
        {'type_': vision.Feature.Type.FACE_DETECTION},
        {'type_': vision.Feature.Type.LABEL_DETECTION},
        {'type_': vision.Feature.Type.TEXT_DETECTION}  
        ]
    })

    # The API response is a protobuf object, which is not JSON serializable.
    # So we need to convert it to a JSON serializable object.
    # Solution from https://stackoverflow.com/a/65728119
    response_json = AnnotateImageResponse.to_json(response)

    # The structure of the response is detailed in the API documentation here:
    # https://cloud.google.com/vision/docs/reference/rest/v1/AnnotateImageResponse
    # The various bits are detailed for each feature type.
    # Here's the documentation for entity annotations, with a link to the BoundingPoly object.
    # https://cloud.google.com/vision/docs/reference/rest/v1/AnnotateImageResponse#EntityAnnotation
    response_struct = json.loads(response_json)

    # Object localization
    # -------------------

    for annotation in response_struct['localizedObjectAnnotations']:
        row = extract_object_localization_data(accession_number, annotation, width, height)
        object_localization_dataframe = object_localization_dataframe.append(row, ignore_index=True)
    
    # Write the annotations to a CSV file after every image in case the process is interrupted.
    object_localization_dataframe.to_csv(base_path + 'object_localization.csv', index=False)
    
    # Face detection
    # --------------
    '''
    analysis_type = vision.Feature.Type.FACE_DETECTION
    response = client.annotate_image({
    'image': {'source': {'image_uri': image_uri}},
    'features': [{'type_': analysis_type}]
    })
    response_json = AnnotateImageResponse.to_json(response)
    response_struct = json.loads(response_json)
    '''
    for annotation in response_struct['faceAnnotations']:
        row = extract_face_detection_data(accession_number, annotation, width, height)
        face_detection_dataframe = face_detection_dataframe.append(row, ignore_index=True)
    
    # Write the annotations to a CSV file after every image in case the process is interrupted.
    face_detection_dataframe.to_csv(base_path + 'face_detection.csv', index=False)
    
    # Label detection
    # ---------------
    '''
    analysis_type = vision.Feature.Type.LABEL_DETECTION
    response = client.annotate_image({
    'image': {'source': {'image_uri': image_uri}},
    'features': [{'type_': analysis_type}]
    })
    response_json = AnnotateImageResponse.to_json(response)
    response_struct = json.loads(response_json)
    # print(json.dumps(response_struct, indent=2))
    '''
    for annotation in response_struct['labelAnnotations']:
        row = extract_label_detection_data(accession_number, annotation)
        label_detection_dataframe = label_detection_dataframe.append(row, ignore_index=True)
    
    # Write the annotations to a CSV file after every image in case the process is interrupted.
    label_detection_dataframe.to_csv(base_path + 'label_detection.csv', index=False)
    
    # Text detection
    # --------------
    '''
    analysis_type = vision.Feature.Type.TEXT_DETECTION
    response = client.annotate_image({
    'image': {'source': {'image_uri': image_uri}},
    'features': [{'type_': analysis_type}]
    })
    response_json = AnnotateImageResponse.to_json(response)
    response_struct = json.loads(response_json)
    #print(json.dumps(response_struct, indent=2))
    '''
    for annotation in response_struct['textAnnotations']:
        row = extract_text_detection_data(accession_number, annotation, width, height)
        text_detection_dataframe = text_detection_dataframe.append(row, ignore_index=True)

    # Write the annotations to a CSV file after every image in case the process is interrupted.
    text_detection_dataframe.to_csv(base_path + 'text_detection.csv', index=False)

print('done')



# Create IIIF annotation file

To create the annotations, we need to convert the relative dimensions to the absolute pixel dimensions based on the canvas size.

The canvas size is given as the dimensions of the full-sized image, which is reported as `max_height` and `max_width` in the dimensions CSV.

In [ ]:
# This code is part of google_cloud_vision.ipynb
# For licensing and other information, see https://github.com/HeardLibrary/linked-data/tree/master/image_analysis

# object_localization.csv contains the results of the object localization analysis
object_localization_dataframe = pd.read_csv(base_path + 'object_localization.csv')
# accession_dimensions.csv is a temporary file that contains the dimensions of the full-size images (max_height and max_width)
# as retrieved from the IIIF manifest
accession_dataframe = pd.read_csv(base_path + 'accession_dimensions.csv', dtype=str)

# Loop through each accession number and create an annotation for each localized object.
for image_index, image_row in accession_dataframe.iterrows():
    print('Processing image ' + str(image_index + 1) + ' of ' + str(len(accession_dataframe)))
    # Build the resources list for the annotations.
    resources = []
    
    # Loop through each object in the image.
    for object_index, object_row in object_localization_dataframe.iterrows():
        if object_row['accession_number'] != image_row['accession_number']:
            continue

        # Create a W3C fragment selector for the annotation.
        # https://www.w3.org/TR/annotation-model/#fragment-selector
        # Calculate the upper left x and y in absolute canvas coordinates.
        x = str(round(object_row['rel_left_x'] * float(image_row['max_width'])))
        y = str(round(object_row['rel_top_y'] * float(image_row['max_height'])))

        # Calculate the width and height in absolute canvas coordinates.
        width = str(round((object_row['rel_right_x'] - object_row['rel_left_x']) * float(image_row['max_width'])))
        height = str(round((object_row['rel_bottom_y'] - object_row['rel_top_y']) * float(image_row['max_height'])))

        fragment_selector = 'xywh=' + x + ',' + y + ',' + width + ',' + height

        # Build the annotation.
        on_value = {
            '@type': 'oa:SpecificResource',
            'full': 'https://iiif-manifest.library.vanderbilt.edu/gallery/' + image_row['accession_number'].split('.')[0] + '/' + image_row['accession_number'] + '.json_1',
            'selector': {
                'type': 'oa:FragmentSelector',
                'value': fragment_selector
            },
            'within': {
                '@id': 'https://iiif-manifest.library.vanderbilt.edu/gallery/' + image_row['accession_number'].split('.')[0] + '/' + image_row['accession_number'] + '.json',
                '@type': 'sc:Manifest'
            }
        }
        resource_value = {
            '@type': 'dctypes:Text',
            'format': 'text/plain',
            'chars': object_row['description']
        }

        annotation = {
            '@context': 'http://iiif.io/api/presentation/2/context.json',
            '@id': 'https://iiif-manifest.library.vanderbilt.edu/gallery/' + image_row['accession_number'].split('.')[0] + '/' + image_row['accession_number'] + '/annotation/' + str(object_index),
            '@type': 'oa:Annotation',
            'motivation': [
                'oa:commenting'
            ],
            'on': on_value,
            'resource': [
                resource_value
            ]
        }
        resources.append(annotation)
    
    annotations = {
        "@context": "http://www.shared-canvas.org/ns/context.json",
        "@id": annotations_base_url + image_row['accession_number'].split('.')[0] + "/" + image_row['accession_number'] + "_annotations.json",
        "@type": "sc:AnnotationList",
        "resources": resources
    }

    # Write the annotations to a JSON file.
    with open(base_path + 'annotations/' + image_row['accession_number'] + '_annotations.json', 'w') as outfile:
        output_text = json.dumps(annotations, indent=2)
        outfile.write(output_text)

print('done')



# Add the link from the manifest to the annotation URL

The annotations file can't be applied to the manifest unless the manifest has a link to it's web address. So an `otherContent` link must be added to the canvas that's being annotated. The link URL has to be a real URL that dereferences, since the annotations have to be retrieved on the fly when the viewer applies the annotations to the canvas.

In [ ]:
# Step through each image in the accession dimensions CSV file.
for image_index, image_row in accession_dataframe.iterrows():
    print(image_row['accession_number'])

    # Look up the manifest URL for the image in the source image dataframe.
    manifest_url = source_image_dataframe.loc[source_image_dataframe['accession_number'] == image_row['accession_number'], 'iiif_manifest'].iloc[0]
    
    # Get the manifest JSON.
    manifest_response = requests.get(manifest_url)
    manifest_json = manifest_response.json()
    
    # Create otherContent dictionary.
    other_content = [
        {
        '@id': annotations_base_url + image_row['accession_number'].split('.')[0] + "/" + image_row['accession_number'] + "_annotations.json",
        '@type': 'sc:AnnotationList'
        }
    ]

    # Add the otherContent dictionary to the manifest.
    manifest_json['sequences'][0]['canvases'][0]['otherContent'] = other_content

    # Write the manifest to a JSON file.
    with open(base_path + 'manifests/' + image_row['accession_number'] + '.json', 'w') as outfile:
        text = json.dumps(manifest_json, indent=4)
        outfile.write(text)

print('done')



# Concatenate the DataFrames

In [ ]:
import os
import pandas as pd

base_path = '/Users/baskausj/github/vandycite/gallery_buchanan/image_analysis/'

# Get the directory names for the subdirectories of the output directory.
subdirectories = os.listdir(base_path + 'output/')
subdirectories.remove('.DS_Store')
# Sort the subdirectories ascending.
subdirectories.sort()

# Create dataframes for each analysis type.
face_detection_dataframe = pd.DataFrame()
label_detection_dataframe = pd.DataFrame()
text_detection_dataframe = pd.DataFrame()
object_localization_dataframe = pd.DataFrame()

# Loop through each subdirectory, load each CSV file, and concatenate the face_detection.csv files into a single CSV file.
for subdirectory in subdirectories:
    print('Processing subdirectory ' + subdirectory)
    # Load the CSV files.
    face_detection_dataframe = face_detection_dataframe.append(pd.read_csv(base_path + 'output/' + subdirectory + '/face_detection.csv', dtype=str))
    label_detection_dataframe = label_detection_dataframe.append(pd.read_csv(base_path + 'output/' + subdirectory + '/label_detection.csv', dtype=str))
    text_detection_dataframe = text_detection_dataframe.append(pd.read_csv(base_path + 'output/' + subdirectory + '/text_detection.csv', dtype=str))
    object_localization_dataframe = object_localization_dataframe.append(pd.read_csv(base_path + 'output/' + subdirectory + '/object_localization.csv', dtype=str))

# Write the concatenated CSV files.
face_detection_dataframe.to_csv(base_path + 'output/face_detection.csv', index=False)
label_detection_dataframe.to_csv(base_path + 'output/label_detection.csv', index=False)
text_detection_dataframe.to_csv(base_path + 'output/text_detection.csv', index=False)
object_localization_dataframe.to_csv(base_path + 'output/object_localization.csv', index=False)

print('done')


# Create a table of detected object labels and the IIIF server URLs that would display the bounded regions

This code is a hack of the `Create IIIF annotation file` cell in this notebook.



In [ ]:
# This code is part of google_cloud_vision.ipynb
# For licensing and other information, see https://github.com/HeardLibrary/linked-data/tree/master/image_analysis

# object_localization.csv contains the results of the object localization analysis
object_localization_dataframe = pd.read_csv(base_path + 'object_localization.csv')
# accession_dimensions.csv is a temporary file that contains the dimensions of the full-size images (max_height and max_width)
# as retrieved from the IIIF manifest
accession_dataframe = pd.read_csv(base_path + 'accession_dimensions.csv', dtype=str)

# Open the download_image_data.csv file. This file is needed to associate the accession numbers with the file names (i.e. to get the file extension).
download_image_data_df = pd.read_csv(base_path + 'download_image_data.csv', dtype=str, na_filter=False)

# For testing, just use the first 10 rows of the dataframe
#accession_dataframe = accession_dataframe.head(10)

# Create a dataframe to hold the data on detected objects.
object_localization_output_dataframe = pd.DataFrame(columns=['accession_number', 'status', 'description', 'image_url'])


# Loop through each accession number and create an row for each localized object.
for image_index, image_row in accession_dataframe.iterrows():
    print()
    print('Processing image ' + str(image_index + 1) + ' of ' + str(len(accession_dataframe)))

    # Extract the year from the first part of the accession number (before the period).
    year = image_row['accession_number'].split('.')[0]

    # Look up the local filename of the image from the download_image_data dataframe.
    # NOTE: This assumes that there is a single image for each local identifier (i.e. accession number).
    local_filename = download_image_data_df.loc[download_image_data_df['local_identifier'] == image_row['accession_number'], 'local_filename'].iloc[0]

    # Create the base IIIF 3.0 URL including the server and identifier components.
    base_url = 'https://iiif.library.vanderbilt.edu/iiif/2/gallery%2F' + year + '%2F' + local_filename + '/'

    print('base_url', base_url)

    # Perform an HTTP GET to see if the image exists on the IIIF server
    response = requests.get(base_url + 'info.json')
    if response.status_code == 200:
        status = 'uploaded'
    else:
        status = 'local'
    print(status)
    
    # Loop through each object in the image.
    for object_index, object_row in object_localization_dataframe.iterrows():
        if object_row['accession_number'] != image_row['accession_number']:
            continue
        '''
        # Create a W3C fragment selector for the annotation.
        # https://www.w3.org/TR/annotation-model/#fragment-selector
        # Calculate the upper left x and y in absolute canvas coordinates.
        x = str(round(object_row['rel_left_x'] * float(image_row['max_width'])))
        y = str(round(object_row['rel_top_y'] * float(image_row['max_height'])))
        '''

        # Set the x and y to be percents of the full image width and height.
        x = str(object_row['rel_left_x'] * 100)
        y = str(object_row['rel_top_y'] * 100)

        '''
        # Calculate the width and height in absolute canvas coordinates.
        width = str(round((object_row['rel_right_x'] - object_row['rel_left_x']) * float(image_row['max_width'])))
        height = str(round((object_row['rel_bottom_y'] - object_row['rel_top_y']) * float(image_row['max_height'])))
        '''

        # Set the width and height to be percents of the full image width and height.
        width = str((object_row['rel_right_x'] - object_row['rel_left_x']) * 100)
        height = str((object_row['rel_bottom_y'] - object_row['rel_top_y']) * 100)

        # Substitute the pct: string for the absolute fragment selector.
        #fragment_selector = 'xywh=' + x + ',' + y + ',' + width + ',' + height
        selector = 'pct:' + x + ',' + y + ',' + width + ',' + height

        # Add the selector to the base URL to specify the region of the image.
        url = base_url + selector + '/full/0/default.jpg'
        print('url', url)
        print('description', object_row['description'])

        # Add the accession number, description, and image URL to the dataframe.
        object_localization_output_dataframe = object_localization_output_dataframe.append({'accession_number': image_row['accession_number'], 'status': status, 'description': object_row['description'], 'image_url': url}, ignore_index=True)

    # Write the dataframe to a CSV file.
    object_localization_output_dataframe.to_csv(base_path + 'object_localization_image_urls.csv', index=False)

print('done')